In [53]:
import os

import json
import pandas as pd
import numpy as np
import aiohttp
import asyncio
from dotenv import load_dotenv
import re

load_dotenv()

KAMIS_CERT_KEY = os.getenv('KAMIS_CERT_KEY')
KAMIS_CERT_ID = os.getenv('KAMIS_CERT_ID')
CATEGORY_CODES = ['100', '200', '300', '400']

In [54]:
# 18-01-01 ~ 23-12-31
# 사용할 작물별로 그날의 평균값
# 10일 단위로 최대 최소 가격
# 10일 단위 전체 평균 가격

In [55]:
# api 호출
async def get_daily_price(date: str, category_code: str,
    session: aiohttp.client.ClientSession) -> json:
    headers = {'Content-Type': 'application/json', 'charset': 'UTF-8', 'Accept': '*/*'}
    request_url = 'http://www.kamis.or.kr/service/price/xml.do?action=dailyPriceByCategoryList'
    params = {
        'p_cert_key': KAMIS_CERT_KEY,
        'p_cert_id': KAMIS_CERT_ID,
        'p_item_category_code': category_code,
        'p_regday': date,
        'p_returntype': 'json'
    }

    for param in params.keys():
        request_url += f"&{param}={params[param]}"

    try:
        async with session.get(request_url) as response:
            return await response.json(content_type='text/plain')
    except Exception as e:
        return await get_daily_price(date, category_code, session)

In [60]:
# 하루동안 모든 카테고리에 속하는 작물의 모든 등급에 대한 가격 평균
async def get_daily_product_cost(date: str) -> dict:
    product_acc_cost = {}
    tasks = []
    async with aiohttp.ClientSession(timeout=aiohttp.ClientTimeout(20)) as session:
        for cat in CATEGORY_CODES:
            task = asyncio.create_task(get_daily_price(date, cat, session))
            tasks.append(task)

        response = await asyncio.gather(*tasks)

        for res in response:
            data = res['data']

            # 비정상 응답, error_code - 001, 200, 900
            if type(data) == list:
                continue

            for item in data['item']:
                item_code = int(item['item_code'])
                product_cost = item['dpr1'].replace(',', '')
                unit_text = item['unit']
                match = re.match(r'([\d.]+)(\D+)', re.sub(r'\([^)]*\)', '', unit_text))
                number = match.group(1)
                unit = match.group(2)

                if product_cost == '-':
                    continue

                product_cost = int(product_cost)
                
                if product_acc_cost.get(item_code) is None:
                    product_acc_cost[item_code] = {
                        'total_cost': 0,
                        'total_count': 0,
                        'unit': ''
                    }

                product_acc_cost[item_code]['total_cost'] += product_cost / float(number)
                product_acc_cost[item_code]['total_count'] += 1
                product_acc_cost[item_code]['unit'] = unit

    result = {'date': date}
    product_avg_cost = {}

    for key in product_acc_cost.keys():
        acc_cost = product_acc_cost.get(key)
        product_avg_cost[key] = {
            'daily_cost': acc_cost['total_cost'] / acc_cost['total_count'],
            'unit': acc_cost['unit']
        }

    result["cost"] = product_avg_cost
    return result

In [61]:
async def get_monthly_product_cost_df(year: str, month: str) -> pd.DataFrame:
    div, mod = divmod(int(month), 12)
    tasks = []
    for date in pd.date_range(f'{year}-{month}-01', f'{int(year) + div}-{mod + 1}-01',
                              inclusive='left').strftime('%Y-%m-%d'):
        task = asyncio.create_task(get_daily_product_cost(date))
        tasks.append(task)

    daily_product_cost_list = await asyncio.gather(*tasks)

    result = pd.DataFrame(
        columns=['YEAR', 'COUNT_PER_TEN_DAYS', 'PRODUCT_CODE', 'MIN_PRODUCT_COST',
                 'MAX_PRODUCT_COST', 'AVG_PRODUCT_COST', 'UNIT']).astype({
        'YEAR': 'int',
        'COUNT_PER_TEN_DAYS': 'int',
        'PRODUCT_CODE': 'str',
        'MIN_PRODUCT_COST': 'float',
        'MAX_PRODUCT_COST': 'float',
        'AVG_PRODUCT_COST': 'float',
        'UNIT': 'str'
    })

    first_third = {}
    second_third = {}
    final_third = {}

    for product_cost in daily_product_cost_list:
        date = pd.to_datetime(product_cost['date'])
        cost = product_cost['cost']
        day = date.day

        target_third = None

        if day <= 10:
            target_third = first_third
        elif day <= 20:
            target_third = second_third
        else:
            target_third = final_third

        for key in cost.keys():
            if target_third.get(key) is None:
                target_third[key] = {
                    'MIN_PRODUCT_COST': float('inf'),
                    'MAX_PRODUCT_COST': 0,
                    'TOTAL_AVG_PRODUCT_COST': 0,
                    'TOTAL_COUNT': 0,
                    'UNIT': ''
                }

            daily_info = cost[key]
            target_third[key]['MIN_PRODUCT_COST'] = np.min(
                [target_third[key]['MIN_PRODUCT_COST'], daily_info['daily_cost']])
            target_third[key]['MAX_PRODUCT_COST'] = np.max(
                [target_third[key]['MAX_PRODUCT_COST'], daily_info['daily_cost']])
            target_third[key]['TOTAL_AVG_PRODUCT_COST'] += daily_info['daily_cost']
            target_third[key]['TOTAL_COUNT'] += 1
            target_third[key]['UNIT'] = daily_info['unit']

    for key in first_third.keys():
        info = first_third[key]
        result = pd.concat([result, pd.DataFrame({
            'YEAR': [int(year)],
            'COUNT_PER_TEN_DAYS': [(int(month) - 1) * 3],
            'PRODUCT_CODE': [key],
            'MIN_PRODUCT_COST': [info['MIN_PRODUCT_COST']],
            'MAX_PRODUCT_COST': [info['MAX_PRODUCT_COST']],
            'AVG_PRODUCT_COST': [info['TOTAL_AVG_PRODUCT_COST'] / info['TOTAL_COUNT']],
            'UNIT': info['UNIT']
        })], ignore_index=True)

    for key in second_third.keys():
        info = second_third[key]
        result = pd.concat([result, pd.DataFrame({
            'YEAR': [int(year)],
            'COUNT_PER_TEN_DAYS': [(int(month) - 1) * 3 + 1],
            'PRODUCT_CODE': [key],
            'MIN_PRODUCT_COST': [info['MIN_PRODUCT_COST']],
            'MAX_PRODUCT_COST': [info['MAX_PRODUCT_COST']],
            'AVG_PRODUCT_COST': [info['TOTAL_AVG_PRODUCT_COST'] / info['TOTAL_COUNT']],
            'UNIT': info['UNIT']
        })], ignore_index=True)

    for key in final_third.keys():
        info = final_third[key]
        result = pd.concat([result, pd.DataFrame({
            'YEAR': [int(year)],
            'COUNT_PER_TEN_DAYS': [(int(month) - 1) * 3 + 2],
            'PRODUCT_CODE': [key],
            'MIN_PRODUCT_COST': [info['MIN_PRODUCT_COST']],
            'MAX_PRODUCT_COST': [info['MAX_PRODUCT_COST']],
            'AVG_PRODUCT_COST': [info['TOTAL_AVG_PRODUCT_COST'] / info['TOTAL_COUNT']],
            'UNIT': info['UNIT']
        })], ignore_index=True)

    print(f'{year}-{month} done')
    return result


In [62]:
async def get_yearly_product_cost_df(year: str) -> pd.DataFrame:
    tasks = []

    # for month in range(1, 7):
    #     task = asyncio.create_task(get_monthly_product_cost_df(year, str(month)))
    #     tasks.append(task)
    # first_half = await asyncio.gather(*tasks)
    # 
    # await asyncio.sleep(10)
    # 
    # tasks = []
    # 
    # for month in range(7, 13):
    #     task = asyncio.create_task(get_monthly_product_cost_df(year, str(month)))
    #     tasks.append(task)
    #     
    # last_half = await asyncio.gather(*tasks)

    monthly_product_cost_list = []

    for month in range(1, 13):
        # task = asyncio.create_task(get_monthly_product_cost_df(year, str(month)))
        # tasks.append(task)
        monthly_product_cost = await get_monthly_product_cost_df(year, str(month))
        monthly_product_cost_list.append(monthly_product_cost)

    # monthly_product_cost_list = await asyncio.gather(*tasks)

    result = pd.DataFrame(
        columns=['YEAR', 'COUNT_PER_TEN_DAYS', 'PRODUCT_CODE', 'MIN_PRODUCT_COST',
                 'MAX_PRODUCT_COST', 'AVG_PRODUCT_COST', 'UNIT']).astype({
        'YEAR': 'int',
        'COUNT_PER_TEN_DAYS': 'int',
        'PRODUCT_CODE': 'str',
        'MIN_PRODUCT_COST': 'float',
        'MAX_PRODUCT_COST': 'float',
        'AVG_PRODUCT_COST': 'float',
        'UNIT': 'str'
    })

    # monthly_product_cost = [*first_half, *last_half]

    result = pd.concat([result, *monthly_product_cost_list], ignore_index=True)
    return result

In [63]:
product_cost_2018 = await get_yearly_product_cost_df('2018')
product_cost_2019 = await get_yearly_product_cost_df('2019')
product_cost_2020 = await get_yearly_product_cost_df('2020')
product_cost_2021 = await get_yearly_product_cost_df('2021')
product_cost_2022 = await get_yearly_product_cost_df('2022')
product_cost_2023 = await get_yearly_product_cost_df('2023')


2018-1 done
2018-2 done
2018-3 done
2018-4 done
2018-5 done
2018-6 done
2018-7 done
2018-8 done
2018-9 done
2018-10 done
2018-11 done
2018-12 done
2019-1 done
2019-2 done
2019-3 done
2019-4 done
2019-5 done
2019-6 done
2019-7 done
2019-8 done
2019-9 done
2019-10 done
2019-11 done
2019-12 done
2020-1 done
2020-2 done
2020-3 done
2020-4 done
2020-5 done
2020-6 done
2020-7 done
2020-8 done
2020-9 done
2020-10 done
2020-11 done
2020-12 done
2021-1 done
2021-2 done
2021-3 done
2021-4 done
2021-5 done
2021-6 done
2021-7 done
2021-8 done
2021-9 done
2021-10 done
2021-11 done
2021-12 done
2022-1 done
2022-2 done
2022-3 done
2022-4 done
2022-5 done
2022-6 done
2022-7 done
2022-8 done
2022-9 done
2022-10 done
2022-11 done
2022-12 done
2023-1 done
2023-2 done
2023-3 done
2023-4 done
2023-5 done
2023-6 done
2023-7 done
2023-8 done
2023-9 done
2023-10 done
2023-11 done
2023-12 done


In [64]:
product_cost_2018.to_csv("data/product_cost_2018.csv", encoding='utf-8')
product_cost_2019.to_csv("data/product_cost_2019.csv", encoding='utf-8')
product_cost_2020.to_csv("data/product_cost_2020.csv", encoding='utf-8')
product_cost_2021.to_csv("data/product_cost_2021.csv", encoding='utf-8')
product_cost_2022.to_csv("data/product_cost_2022.csv", encoding='utf-8')
product_cost_2023.to_csv("data/product_cost_2023.csv", encoding='utf-8')

In [65]:
product_cost_2017 = await get_monthly_product_cost_df('2017', '12')
product_cost_2017 = product_cost_2017.loc[product_cost_2017.COUNT_PER_TEN_DAYS == 35, 'YEAR':]
product_cost_2017.to_csv("data/product_cost_2017.csv", encoding='utf-8')

2017-12 done


In [77]:
product_cost = pd.concat([product_cost_2017, product_cost_2018, product_cost_2019, product_cost_2020, product_cost_2021, product_cost_2022, product_cost_2023])

In [78]:
basic_product_info = pd.read_csv('data/basic_product_info.csv', encoding='utf-8')

In [79]:
selected_basic_product_info = basic_product_info.loc[basic_product_info['PRODUCT_CODE'].isin(product_cost['PRODUCT_CODE'].unique().tolist())]

In [80]:
unit_info = product_cost[['PRODUCT_CODE', 'UNIT']]


In [81]:
unit_info = unit_info.drop_duplicates(ignore_index=True)
unit_info = unit_info.drop([55])

In [82]:
unit_info

,PRODUCT_CODE,UNIT
0,111,kg
1,112,kg
2,141,kg
3,142,kg
4,143,kg
5,144,kg
6,151,kg
7,152,kg
8,211,kg
9,212,kg


In [83]:
selected_basic_product_info = pd.merge(left=selected_basic_product_info, right=unit_info, on='PRODUCT_CODE')

In [84]:
selected_basic_product_info

,PRODUCT_CODE,PRODUCT_NAME,M_DISTCTNS,PRDCTN_ERA,UNIT
0,111,쌀,FALL,1536,kg
1,112,찹쌀,FALL,1536,kg
2,141,콩,ALL,0,kg
3,142,팥,FALL,1536,kg
4,143,녹두,FALL,1536,kg
5,144,메밀,FALL,3072,kg
6,151,고구마,FALL,1536,kg
7,152,감자,SUMMER,1984,kg
8,211,배추,FALL,6144,kg
9,212,양배추,SPRING,224,kg


In [86]:
selected_basic_product_info.to_csv('data/selected_basic_product_info.csv', encoding='utf-8', index=False)